In [1]:
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
from catboost import CatBoostClassifier,Pool
from metric import decision_prices

Загружаем тестовую выборку

In [2]:
test = pd.read_excel('test.xlsx')
test

,dt,Цена на арматуру
0,2022-09-05,46000
1,2022-09-12,46000
2,2022-09-19,46000
3,2022-09-26,45500
4,2022-10-03,45500
5,2022-10-10,45000
6,2022-10-17,44000
7,2022-10-24,43000
8,2022-10-31,42000
9,2022-11-07,41500


In [3]:
test['Data_Time'] = pd.to_datetime(test.dt)
test['quarter'] = test['Data_Time'].dt.quarter
test['month'] = test['Data_Time'].dt.month
test['year'] = test['Data_Time'].dt.year
test['dayofyear'] = test['Data_Time'].dt.dayofyear
test['dayofmonth'] = test['Data_Time'].dt.day
test['weekofyear'] = test['Data_Time'].dt.isocalendar().week

# Делаем поля времени трейна категориальными
for col in ['quarter',
       'month',  'weekofyear']:
    test[col] = test[col].astype('category')
# набор признаков времени на вход
columns_to_train=['quarter',
       'month',  'year', 'dayofyear', 'dayofmonth', 'weekofyear']
test

,dt,Цена на арматуру,Data_Time,quarter,month,year,dayofyear,dayofmonth,weekofyear
0,2022-09-05,46000,2022-09-05,3,9,2022,248,5,36
1,2022-09-12,46000,2022-09-12,3,9,2022,255,12,37
2,2022-09-19,46000,2022-09-19,3,9,2022,262,19,38
3,2022-09-26,45500,2022-09-26,3,9,2022,269,26,39
4,2022-10-03,45500,2022-10-03,4,10,2022,276,3,40
5,2022-10-10,45000,2022-10-10,4,10,2022,283,10,41
6,2022-10-17,44000,2022-10-17,4,10,2022,290,17,42
7,2022-10-24,43000,2022-10-24,4,10,2022,297,24,43
8,2022-10-31,42000,2022-10-31,4,10,2022,304,31,44
9,2022-11-07,41500,2022-11-07,4,11,2022,311,7,45


Считываем модель

In [4]:
best_model = CatBoostClassifier()
best_model.load_model('catboost_model.bin')


In [5]:
test['Prognoz']=best_model.predict(test[columns_to_train])
test

,dt,Цена на арматуру,Data_Time,quarter,month,year,dayofyear,dayofmonth,weekofyear,Prognoz
0,2022-09-05,46000,2022-09-05,3,9,2022,248,5,36,больше
1,2022-09-12,46000,2022-09-12,3,9,2022,255,12,37,больше
2,2022-09-19,46000,2022-09-19,3,9,2022,262,19,38,меньше
3,2022-09-26,45500,2022-09-26,3,9,2022,269,26,39,меньше
4,2022-10-03,45500,2022-10-03,4,10,2022,276,3,40,больше
5,2022-10-10,45000,2022-10-10,4,10,2022,283,10,41,больше
6,2022-10-17,44000,2022-10-17,4,10,2022,290,17,42,меньше
7,2022-10-24,43000,2022-10-24,4,10,2022,297,24,43,больше
8,2022-10-31,42000,2022-10-31,4,10,2022,304,31,44,меньше
9,2022-11-07,41500,2022-11-07,4,11,2022,311,7,45,меньше


In [6]:
# Здесь нужно вставка алгоритма!

test.loc[::2,'Объем'] = 2 
test['Объем'] = test['Объем'].fillna(0)

In [7]:
test

,dt,Цена на арматуру,Data_Time,quarter,month,year,dayofyear,dayofmonth,weekofyear,Prognoz,Объем
0,2022-09-05,46000,2022-09-05,3,9,2022,248,5,36,больше,2.0
1,2022-09-12,46000,2022-09-12,3,9,2022,255,12,37,больше,0.0
2,2022-09-19,46000,2022-09-19,3,9,2022,262,19,38,меньше,2.0
3,2022-09-26,45500,2022-09-26,3,9,2022,269,26,39,меньше,0.0
4,2022-10-03,45500,2022-10-03,4,10,2022,276,3,40,больше,2.0
5,2022-10-10,45000,2022-10-10,4,10,2022,283,10,41,больше,0.0
6,2022-10-17,44000,2022-10-17,4,10,2022,290,17,42,меньше,2.0
7,2022-10-24,43000,2022-10-24,4,10,2022,297,24,43,больше,0.0
8,2022-10-31,42000,2022-10-31,4,10,2022,304,31,44,меньше,2.0
9,2022-11-07,41500,2022-11-07,4,11,2022,311,7,45,меньше,0.0


In [ ]:
test=test['dt','Цена на арматуру','Объем']

In [6]:
test.to_excel('submission.xlsx', index=False)

Считаем затраты на арматуру

In [7]:
test_submission = pd.read_excel('submission.xlsx')

In [8]:
decision_prices(test_submission)

1236800